## Pre-processing methods and decomposition models

Useful methods for pre-processing the galaxy object to perform the dynamical decomposition:

 - Center funtion: `preproc.center(galaxy)`
 - Star align function : `preproc.star_alingn(galaxy)`
 - Circular angular momentum function : `preproc.jcirc(galaxy)`


In [1]:
import galaxychop as gchop

gal = gchop.io.read_hdf5("gal394242.h5")
gal

<Galaxy stars=32067, dark_matter=21156, gas=4061, potential=True>

The packages provides two function to validate if a galaxy is centered and aligned:

In [2]:
print("Centered:", gchop.preproc.is_centered(gal))
print("Aligned:", gchop.preproc.is_star_aligned(gal))

Centered: False
Aligned: False


First of all, to perform the dynamical decomposition is necesary center the position of all galaxy particles respect to the position of the lowest potential particle.

In [3]:
gal = gchop.preproc.center(gal)

Secondly, the position particles are rotatated so that the total angular moment of the stars particles coincides
with the z-axis.

Optionally, only stars particles within a cutting radius `(r_cut)` can be used to calculate the rotation matrix.

In [4]:
gal = gchop.preproc.star_align(gal)

In [5]:
print("Centered:", gchop.preproc.is_centered(gal))
print("Aligned:", gchop.preproc.is_star_aligned(gal))
print("Centered & Aligned:", gchop.preproc.is_centered_and_aligned(gal))

Centered: True
Aligned: True
Centered & Aligned: True


Finally, the function `jcirc` calculate the normalized specific energy of the stars, circularity parameter, projected circularity parameter and the points: x = E_star_norm and y = J_circ(E_star_norm), to build the circular angular momentum function for stellar particles.

- normalized_star_energy: Normalized specific energy of stars
- eps: Circularity parameter (J_z/J_circ)
- eps_r: Projected circularity parameter (J_p/J_circ)
- x: Normalized specific energy for the particle with the maximum z-component of the normalized specific angular momentum per bin.
- y: Maximum value of the z-component of the normalized specific angular momentum per bin.

To perform the calculus of `x` and `y` per bin, the size of bins can be included.

In [6]:
cric_prop = gchop.preproc.jcirc(gal, bin0=0.05, bin1=0.005)

print(cric_prop.normalized_star_energy)
print(cric_prop.normalized_star_Jz)
print(cric_prop.eps)
print(cric_prop.eps_r)

[-0.99509357 -0.99760334 -0.99279739 ... -0.13989039 -0.08796932
 -0.06652457]
[-9.24079355e-05  1.56582528e-04 -3.68821838e-04 ... -1.37554847e-01
  4.23475904e-01  7.12919724e-01]
[-0.10974301  0.38068977 -0.29837353 ... -0.17532475  0.54943866
  0.86226812]
[0.8643671  1.15379305 0.53883725 ... 0.53970452 1.12444831 0.3601765 ]


<div class="alert alert-info">
Note: jcirc
    
This calculus is necessary only for stars, but by constuction, we calculated it for all partecles.
For more information, you can access the documentation of the jcirc method.
</div>

## Decomposition

To perform the dynamical decomposition the package provides several model:

    - JThreshold
    - JHistogram
    - JEHistogram
    - KMeans
    - GaussianMixture
    - AutoGaussianMixture
    
The procedure to find to which stellar component each star particle belongs is exactly the same for all models:

First instantiate the decomposer

In [7]:
decomposer = gchop.models.JThreshold()
decomposer

JThreshold(cbins=(0.05, 0.005), eps_cut=0.6)

Decompose the galaxy into components

In [8]:
components = decomposer.decompose(gal)
components

<Components length=57284, labels={'Spheroid', nan, 'Disk'}, probabilities=False, lmap=True>

In [9]:
components.labels

array([ 0.,  0.,  0., ..., nan, nan, nan])

In [10]:
components.ptypes

array(['stars', 'stars', 'stars', ..., 'gas', 'gas', 'gas'], dtype='<U11')

In [11]:
components.to_dataframe()